In [1]:
from xml.etree import ElementTree
import numpy as np
from itertools import combinations
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
tmvseqs = ElementTree.parse("./data/seqs/ledger.xml")

In [55]:
tmvseqs.getroot()

<Element 'DOCUMENT_ROOT' at 0x00000247FB924EF0>

In [14]:
seqs = tmvseqs.getroot().findall("RECORD")

In [16]:
for seq in seqs:
    print(seq.find("ACCESSION_NO").text)

OK377342
OK377341
OK377343
OK424595
OK362288


In [19]:
fasta = []

for seq in seqs:
   fasta.append(seq.find("FASTA").text.replace('\n', '').replace('\t', ''))

In [22]:
# All sequences seem to have the same length!

[len(f) for f in fasta]

[2746, 2746, 2746, 2746, 2746]

In [51]:
seqarrays = [np.array([letter for letter in f]) for f in fasta]

In [52]:
seqpairs = list(combinations(seqarrays, r = 2))

In [53]:
# There are differences.

[(sp[0] != sp[1]).sum() for sp in seqpairs]

[37, 44, 12, 5, 7, 39, 32, 42, 39, 7]

In [42]:
# https://ictv.global/report/chapter/geminiviridae/geminiviridae/begomovirus

In [40]:
# Copy pasted the HTML manually from Firefox. _|_ lazy loads.

with open("./data/Genus Begomovirus ICTV.html", mode = "r", encoding = "utf8") as html:
    content = html.read()
    soup = BeautifulSoup(content, "html.parser")

In [67]:
table = soup.find_all("table", attrs = {"class" : "virus-isolates-table"})[0]

In [68]:
len(table)

2

In [87]:
headers = [th.text for th in table.find_all("th")]
headers

['',
 'Genus',
 'Subgenus',
 'Species',
 'Virus name',
 'Isolate',
 'Accession',
 'Available sequence',
 'Abbrev.']

In [88]:
table_dict = {
    "Species": [],
    "Virus name": [],
    "Isolate": [],
    "Accession": [],
    "Available sequence": [],
    "Abbrev":[]
}

In [105]:
[th.text for th in table.find_all("tr")[0].find_all("th")]

['',
 'Genus',
 'Subgenus',
 'Species',
 'Virus name',
 'Isolate',
 'Accession',
 'Available sequence',
 'Abbrev.']

In [108]:
table_dict

{'Species': [],
 'Virus name': [],
 'Isolate': [],
 'Accession': [],
 'Available sequence': [],
 'Abbrev': []}

In [110]:
# Except for the row with table headers, there seems to be two kinds of table rows.
# 1) <tr class="virus-row">
# 2) <tr class="alt-virus-row">

counter = 0
for tr in table.find_all("tr"):
    if counter == 0:
        counter += 1
        continue
    else:
        table_dict["Species"].append(tr.find("td", attrs = {"class": "col-species"}).text)
        table_dict["Virus name"].append(tr.find("td", attrs = {"class": "col-alternativeNameCSV"}).text)
        table_dict["Isolate"].append(tr.find("td", attrs = {"class": "col-isolate"}).text)
        table_dict["Accession"].append(tr.find("td", attrs = {"class": "col-accessionNumber"}).text)
        table_dict["Available sequence"].append(tr.find("td", attrs = {"class": "col-availableSequence"}).text)
        table_dict["Abbrev"].append(tr.find("td", attrs = {"class": "col-abbrev"}).text)
        counter += 1

In [113]:
begomovirus = pd.DataFrame(table_dict)

In [114]:
begomovirus.to_csv("./data/begomovirus_ictv.csv", sep = ',', header = True, index = False, mode = "w")

In [2]:
ictv_begomo = pd.read_csv("./data/begomovirus_ictv.csv")

In [13]:
ind = np.array([True if "cassava" in name.lower() else False for name in ictv_begomo["Virus name"].values])

In [18]:
ictv_begomo.loc[ind, :].to_csv("./data/cassava_filtered.csv", sep = ',', header = True, index = False, mode = "w")

In [3]:
begomo = pd.read_csv("./data/begomovirus_ictv.csv")

In [4]:
begomo.columns

Index(['Species', 'Virus name', 'Isolate', 'Accession', 'Available sequence',
       'Abbrev'],
      dtype='object')

In [11]:
begomo.loc[begomo.Species.str.contains("cassava", case = False), :]

,Species,Virus name,Isolate,Accession,Available sequence,Abbrev
4,African cassava mosaic Burkina Faso virus,African cassava mosaic Burkina Faso virus,BF-Oua-127-08,DNA-A: HE616777;DNA-B: HE616778,Complete genome,ACMBFV
5,African cassava mosaic virus,African cassava mosaic virus,Cameroon/1998,DNA-A: J02057;DNA-B: J02058,Complete genome,ACMV
58,Cassava mosaic Madagascar virus,cassava mosaic Madagascar virus,Madgascar/Toliary/2006,DNA-A: HE617299;DNA-B: HE617300,Complete genome,CMMGV
147,East African cassava mosaic Cameroon virus,East African cassava mosaic Cameroon virus,Cameroon/1998,DNA-A: AF112354;DNA-B: AF112355,Complete genome,EACMCMV
148,East African cassava mosaic Kenya virus,East African cassava mosaic Kenya virus,Kenya/Mitaboni/K298/2002,DNA-A: AJ717572;DNA-B: AJ704971,Complete genome,EACMKV
149,East African cassava mosaic Malawi virus,East African cassava mosaic Malawi virus,Malawi/K/1996,DNA-A: AJ006460,Complete genome,EACMMV
150,East African cassava mosaic virus,East African cassava mosaic virus,Uganda/Severe 2/1997/Uganda,DNA-A: AF126806;DNA-B: AF126807,Complete genome,EACMV/UG
151,East African cassava mosaic virus,East African cassava mosaic virus,Kenya/Boa/K48/2001/Kenya,DNA-A: AJ717542;DNA-B: AJ704949,Complete genome,EACMV/KE
152,East African cassava mosaic virus,East African cassava mosaic virus,Malawi/8N/2007/Malawi,JX473582,Complete genome,EACMV/MW
153,East African cassava mosaic virus,East African cassava mosaic virus,Tanzania/1/2001/Tanzania,DNA-A: AY795983;DNA-B: AY795989,Complete genome,EACMV/TZ


In [12]:
begomo.loc[begomo.Species.str.contains("cassava", case = False), :].shape

(18, 6)

In [ ]:
# https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=fasta&id=89330928&conwithfeat=on&withparts=on&show-sequence=on&hide-cdd=on&ncbi_phid=CE8AC93650A0D641000000000A89088C